# 03: Model - Logistic Regression

## Objective
To implement and evaluate a Logistic Regression model for churn prediction, emphasizing interpretability through coefficients.

## Methodology
1. **Clean Data**: Using `src.preprocessing`.
2. **Training**: Standard Logistic Regression with scaled features.
3. **Interpretability**: Analyze model coefficients.

In [ ]:
# 1. Setup
!pip install -q pandas numpy scikit-learn matplotlib seaborn

import pandas as pd
from src.preprocessing import clean_data

df_raw = pd.read_csv("ecommerce_churn_llm_final.csv")
df = clean_data(df_raw)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

X = df.drop(columns=["customer_id", "churn", "customer_feedback", "support_chat_excerpt", "reason_for_low_activity"], errors="ignore")
y = df["churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

categorical_cols = X.select_dtypes(include=['object', 'string', 'category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['number']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

lr_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)

print(classification_report(y_test, y_pred))
print(f"ROC AUC: {roc_auc_score(y_test, lr_model.predict_proba(X_test)[:, 1]):.3f}")